In [1]:
#looking at the 18 features from >= 4 CIMP+ occurrences
import os
import csv
import statistics

inpdat = "../../../prunedcoadreadDAT2.csv"

with open(inpdat,'r') as f:
    it = csv.reader(f)
    listit = list(it)

print("col # of old:",len(listit[0]))

col # of old: 70


In [2]:
endcol = len(listit[0])-1
endrow = len(listit)

keep = [0]
threeormore3neg = []
model = []
fptot = tptot = 0
covered = [] #covered critical samples
captured = []
seeking = ["KRAS_GRCh37_12:25398284-25398284","BRAF_GRCh37_7:140453136-140453136"]
for y in range(1,endcol):
    TPcount = 0
    FPcount = 0
    for x in range(1,endrow):
        if(float(listit[x][y]) == 1.0):
            if(float(listit[x][endcol]) == 1.0):
                TPcount += 1
            else:
                FPcount += 1
    if(FPcount <= 0):
        fptot += 1
    if(TPcount >= 2):
        tptot += 1
    if (TPcount >= 10 and FPcount <= 5):
        keep.append(y)
        threeormore3neg.append(y) #list of indices we're looking at
        model.append(listit[0][y])
        
    #grab features from seeking
    pieces = listit[0][y].split("_")
    if(pieces[0] + "_" + pieces[1] + "_" + pieces[2] in seeking):
        captured.append(y)
keep.append(len(listit[0])-1)
print(tptot,fptot)
print(str(len(threeormore3neg)))
covered

68 42
2


[]

In [3]:
results = [["Feature","True Positive/CIMP+ Cover","False Postive","True Negative","False Negative","Accuracy"]]
for index in threeormore3neg:
    truepos = trueneg = falsepos = falseneg = 0
    for x in range(1,endrow):
        if(float(listit[x][endcol]) == 1.0):
            if(float(listit[x][index]) == 1.0):
                truepos += 1
            elif(float(listit[x][index]) == 0.0):
                falseneg += 1
        elif(float(listit[x][endcol]) == -1.0):
            if(float(listit[x][index]) == 0.0):
                trueneg += 1
            elif(float(listit[x][index]) == 1.0):
                falsepos += 1
            else:
                print("ERROR")
        else:
            print("ERROR")
    accuracy = 1 - ((falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos))
    newrow = [listit[0][index],truepos,falsepos,trueneg,falseneg,accuracy]
    results.append(newrow)
    
results

[['Feature',
  'True Positive/CIMP+ Cover',
  'False Postive',
  'True Negative',
  'False Negative',
  'Accuracy'],
 ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
  16,
  4,
  38,
  14,
  0.75],
 ['ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----',
  12,
  5,
  37,
  18,
  0.6805555555555556]]

In [4]:
#looking at all features overall
def runGreedyOnTest(model, testdata):
    numrows = len(testdata)
    numcols = len(testdata[0])-1
    results = {}
    for x in range(0, numrows):
        for y in range(0, numcols): #program won't be able to see background/foreground
            if (testdata[x][y] == "1.0"):
                if (testdata[0][y] in model):
                    results[testdata[x][0]] = "1.0"
    for x in range(0,numrows):
        if(testdata[x][0] not in results):
            results[testdata[x][0]] = "-1.0"
    return results
def evalResults(dictofseqs,givendat,picked,notpicked,i):
    falsepos = falseneg = truepos = trueneg = 0
    for x in range (1,len(givendat)):
        if(givendat[x][0] in dictofseqs):
            if(dictofseqs[givendat[x][0]] != givendat[x][len(givendat[0])-1]):
                if(dictofseqs[givendat[x][0]] == "1.0"):
                    falsepos += 1
                else:
                    falseneg += 1
                    if givendat[x][0] not in notpicked:
                        notpicked.append(givendat[x][0])
            else:
                if(dictofseqs[givendat[x][0]] == "1.0"):
                    truepos += 1
                    if givendat[x][0] not in picked:
                        picked.append(givendat[x][0])
                else:
                    trueneg += 1
    tot = len(dictofseqs)
    overallerror = (falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos)
    sens = truepos/(truepos+falseneg)
    spec = trueneg/(trueneg+falsepos)
    k = str(i)
    print("[" + k + "]" + "False Positives: " + str(falsepos))
    print("[" + k + "]" + "False Negatives: " + str(falseneg))
    print("[" + k + "]" + "True Positives: " + str(truepos))
    print("[" + k + "]" + "True Negatives: " + str(trueneg))
    print("[" + k + "]" + "Proportion of false positives: " + str((falsepos/tot)))
    print("[" + k + "]" + "Proportion of false negatives:" + str((falseneg/tot)))
    print("[" + k + "]" + "Accuracy: " + str((1-overallerror)))
    print("[" + k + "]" + "Error rate: " + str(overallerror))
    print("[" + k + "]" + "Sensitivity: " + str(sens))
    print("[" + k + "]" + "Specificity: " + str(spec))
    
picked = []
notpicked = []
dictofseqs = runGreedyOnTest(model,listit)
evalResults(dictofseqs,listit,picked,notpicked,1)
notpicked

[1]False Positives: 8
[1]False Negatives: 10
[1]True Positives: 20
[1]True Negatives: 34
[1]Proportion of false positives: 0.1095890410958904
[1]Proportion of false negatives:0.136986301369863
[1]Accuracy: 0.75
[1]Error rate: 0.25
[1]Sensitivity: 0.6666666666666666
[1]Specificity: 0.8095238095238095


['TCGA-AA-3837',
 'TCGA-AA-3870',
 'TCGA-AA-3941',
 'TCGA-AA-3994',
 'TCGA-AA-A00A',
 'TCGA-AA-A00E',
 'TCGA-AA-A029',
 'TCGA-AA-A02O',
 'TCGA-AA-A03F',
 'TCGA-AG-3575']

In [5]:
#output stats of accuracy filter
with open("./less1.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(results)
    
newdatset = [[row[c] for c in keep] for row in listit]
with open("../../featurefilter/138feats.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(newdatset)